# Exercise 7: Data Streams

## [2 points] How many API credits would be required for downloading 4 years worth of data at 15-minute intervals, per stock?

With the required setup for this assignment, it took 107 API credits to download each stock. However, this was with the 
overlapping method required by this assignment. It would take fewer than 107 credits if this were excluded. 

## [3 points] The API applies a rate limit. How long would it take to download 4 years worth of data at 15-minute intervals, per stock?

With my setup, it took 14 minutes per stock to download the 4 years worth of data with the overlapping technique. My method for gathering the data involved creating chunks of data for each request to be as large as possible to speed up the process. I also made sure to exclude weekends and holidays from being included in these chunks to make the requests faster and more efficient as well. To avoid issues with the API limits, I have a 61 second sleep that automatically triggers if the minutely request limit is reached. I found this method to be faster than some other attempts where I set chunks of 14 or 30 days per request and not checking for holidays or weekends. 

## [20 points for setting up data collection] We’ll be working with just 2 stocks: AAPL and MSFT. The need for speed in a price API makes it inevitable that a small percentage of data elements has errors in it. To compensate for such errors, a “good enough” strategy is to request data for overlapping periods, say fetching 30 minutes of data every 15 minutes – replacing the most recently arrived data with a corrected version.

Below is the setup for using the api to gather the stock data. The full script I ran to gather the data using the API is under the following question.

In [ ]:
# !pip install twelvedata
# import twelvedata

from google.colab import drive
import sys
drive.mount("/content/drive/", force_remount=True)
sys.path.append("/content/drive/My Drive/Colab Notebooks")
from key import twelveDataKey as api_key

## [25 points for collecting data] Technically, final testing can only be done while the market is open: 9:30 am to 4:00 pm on weekdays. In a compromise to accommodate everyone’s schedules we will pull the stock data of each stock and gather up at least 4 years of stock price data (January 2021-December 2024) and use it as the basis of our work.


Below is the script I ran for getting the data and storing them in csv files. I ran the script once each per stock, to allow myself to answer the previous questions about time and API credits per stock more accurately. 

In [ ]:
import time as t
from twelvedata import TDClient
import pandas as pd
from datetime import datetime, timedelta, time
import os
from google.colab import drive
from tqdm import tqdm

# Get key from google drive - avoids embedding in code/uploading to github
td = TDClient(apikey=api_key)
drive.mount('/content/drive')

# API constraints for use in function
MAX_DAILY_REQUESTS = 800
MAX_MINUTE_REQUESTS = 8
DATA_LIMIT_PER_REQUEST = 5000


def fetch_chunk(symbol, start_dt, end_dt, retries=3):
    """Fetch data chunk with retry logic, ensuring API constraints."""
    global api_usage

    if api_usage >= MAX_DAILY_REQUESTS:
        print("Daily API limit reached. Stopping execution.")
        return pd.DataFrame()

    for attempt in range(retries):
        try:
            data = td.time_series(
                symbol=symbol,
                interval="15min",
                start_date=start_dt.strftime("%Y-%m-%d %H:%M:%S"),
                end_date=end_dt.strftime("%Y-%m-%d %H:%M:%S"),
                outputsize=DATA_LIMIT_PER_REQUEST
            ).as_pandas()

            api_usage += 1  # Increment usage counter
            return data[data.index >= start_dt]  # Trim overlap
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            t.sleep(min(8 * (attempt + 1), 60))  # Exponential backoff, capped at 60s

    return pd.DataFrame()

def fetch_stock_data(symbol):
    """Fetch full stock data while accounting for API rate limits."""
    global api_usage

    market_open = time(9, 30)
    market_close = time(16, 0)
    chunk_days = 14
    overlap = timedelta(minutes=30)

    all_data = []
    current_dt = datetime(2021, 1, 1)
    end_dt = datetime(2024, 12, 31)

    # Find first available trading day
    while current_dt <= end_dt:
        if current_dt.weekday() >= 5:  # Skip weekends
            current_dt += timedelta(days=1)
            continue

        test_data = fetch_chunk(
            symbol,
            datetime.combine(current_dt, market_open),
            datetime.combine(current_dt, market_close),
            retries=1
        )
        if not test_data.empty:
            break
        current_dt += timedelta(days=1)

    # Calculate total calendar days
    total_days = (end_dt - current_dt).days + 1  # +1 to include both start and end dates
    if total_days <= 0:
        print(f"No valid data available for {symbol}. Exiting.")
        return

    pbar = tqdm(total=total_days, desc=f"Fetching {symbol}")
    processed_days = 0

    while current_dt <= end_dt and api_usage < MAX_DAILY_REQUESTS and processed_days < total_days:
        # Skip weekends
        if current_dt.weekday() >= 5:
            current_dt += timedelta(days=1)
            pbar.update(1)
            processed_days += 1
            continue

        chunk_end = min(current_dt + timedelta(days=chunk_days - 1), end_dt)  # -1 because we include current day

        chunk_data = fetch_chunk(
            symbol,
            datetime.combine(current_dt, market_open),
            datetime.combine(chunk_end, market_close) + overlap,
        )

        days_in_chunk = (chunk_end - current_dt).days + 1  # +1 to include both start and end dates

        if not chunk_data.empty:
            all_data.append(chunk_data)
            pbar.update(days_in_chunk)
            processed_days += days_in_chunk
            current_dt = chunk_end + timedelta(days=1)  # Move to next day after chunk_end
        else:
            # If no data, move forward by chunk days
            pbar.update(days_in_chunk)
            processed_days += days_in_chunk
            current_dt = chunk_end + timedelta(days=1)

        # API rate limiting. 61 second sleep if hit max minute requests - prevent from going over max
        if api_usage % MAX_MINUTE_REQUESTS == 0:
            t.sleep(61)

    pbar.close()

    if all_data: ##combine data into a csv file to download from cluster
        final_data = pd.concat(all_data)
        final_data = final_data[~final_data.index.duplicated(keep='last')]
        final_data.sort_index(inplace=True)
        save_path = f'/content/drive/My Drive/{symbol}_data.csv'
        final_data.to_csv(save_path)
        print(f"Saved {len(final_data)} records for {symbol} at {save_path}")

    if current_dt > end_dt:
        print(f"Successfully processed all dates for {symbol} ✅")
    elif api_usage >= MAX_DAILY_REQUESTS:
        print(f"Stopped due to reaching API limit at {current_dt.strftime('%Y-%m-%d')} ⚠️")
    else:
        print(f"Unexpected exit condition at {current_dt.strftime('%Y-%m-%d')}")

# Fetch data for both stocks
fetch_stock_data("MSFT")

## [25 points] Create a program new-stock-price-feeder.py that uses a more modern API (e.g., twelveData) instead.

The below code was run with the below command. This was the names of one of my clusters/regions/program name. It was changed when run with different clusters. 
        
        gcloud dataproc jobs submit pyspark stocks2.py --cluster cluster-4290 --region us-east1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, lag, when, lit, input_file_name, round
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("StockTradingStrategy") \
    .getOrCreate()

# Read all CSV files from HDFS put in there before running the script. 
stock_df = spark.read.csv(
    "hdfs:///user/root/stock_data/",
    header=True,
    inferSchema=True
)

# Add symbol column based on filename
stock_df = stock_df.withColumn(
    "filename", input_file_name()
).withColumn(
    "symbol",
    when(
        col("filename").contains("AAPL"), "AAPL"
    ).when(
        col("filename").contains("MSFT"), "MSFT"
    ).otherwise("UNKNOWN")
).drop("filename")

# Verify we have both symbols
print("Distinct symbols found:")
stock_df.select("symbol").distinct().show()

# Define window specs for moving averages
window_10 = Window.partitionBy("symbol").orderBy("datetime").rowsBetween(-9, 0)
window_40 = Window.partitionBy("symbol").orderBy("datetime").rowsBetween(-39, 0)

# Calculate moving averages
stock_with_ma = stock_df.withColumn("ma_10", avg("close").over(window_10)) \
                       .withColumn("ma_40", avg("close").over(window_40))

# Function to generate trading signals with share calculations
def generate_trades(df):
    window_spec = Window.partitionBy("symbol").orderBy("datetime")
    return df.withColumn("prev_ma_10", lag("ma_10", 1).over(window_spec)) \
            .withColumn("prev_ma_40", lag("ma_40", 1).over(window_spec)) \
            .withColumn("signal",
                when(
                    (col("prev_ma_10") <= col("prev_ma_40")) &
                    (col("ma_10") > col("ma_40")), "buy"
                ).when(
                    (col("prev_ma_10") >= col("prev_ma_40")) &
                    (col("ma_10") < col("ma_40")), "sell"
                ).otherwise(None)
            ) \
            .withColumn("shares", round(lit(100000)/col("close"))) \
            .filter(col("signal").isNotNull()) \
            .select(
                "datetime",
                "symbol",
                "close",
                "signal",
                "shares"
            )

# Generate trades
trades_df = generate_trades(stock_with_ma)

# Format output following instructions. 
formatted_trades = trades_df.rdd.map(
    lambda row: f"({row['datetime']} {row['signal']} {row['symbol']}) - {row['shares']} shares @ ${row['close']:.2f}"
).collect()

# Save results to HDFS
trades_df.write.mode("overwrite") \
    .option("header", "true") \
    .csv("hdfs:///user/root/stock_trades/")

# Print for debugging/testing (not necessary)
print("\nTrading Recommendations ($100K per trade):")
for trade in formatted_trades:
    print(trade)

 ## [10 pts] Within Spark, filter the incoming date to create aaplPrice and msftPrice streams.

The code is shown above, but the excerpt is pasted below as well. This chunk of code grabs the csv that is stored in hdfs from the earlier steps.

In [ ]:
# Read all CSV files from HDFS directory
stock_df = spark.read.csv(
    "hdfs:///user/root/stock_data/",
    header=True,
    inferSchema=True
)

## [10 pts] From aaplPrice produce two other streams aapl10Day, aapl40Day. Both of these streams and their comparison to generate buy/sell signals are not shown in the diagram above.

The code for this is shown above in the entire code chunk. The excerpt is pasted below as well.

In [ ]:
window_10 = Window.partitionBy("symbol").orderBy("datetime").rowsBetween(-9, 0)
window_40 = Window.partitionBy("symbol").orderBy("datetime").rowsBetween(-39, 0)

## [10 pts] From msftPrice produce two more streams msft10Day and msft40Day .

 The code is generalized as shown above, so the code for this is the same as for AAPL.

## [20 pts]. Compare the two moving averages (10-day MA and the 40-day MA) to indicate buy and sell signals . Your output should be of the form [( <datetime> buy <symbol>), ( <datetime> sell <symbol>), etc].

Example output from AAPL part of downloaded csv file:

        datetime	               symbol	close	 signal	    shares
        2021-01-04T12:00:00.000Z	AAPL	127.41	sell	785.0
        2021-01-05T10:00:00.000Z	AAPL	131.4893	buy	761.0
        2021-01-06T10:15:00.000Z	AAPL	128.9501	sell	775.0
        2021-01-07T11:30:00.000Z	AAPL	130.705	buy	765.0
        2021-01-11T10:00:00.000Z	AAPL	129.69	sell	771.0
        2021-01-13T10:00:00.000Z	AAPL	129.82001	buy	770.0
        2021-01-14T11:30:00.000Z	AAPL	130.21001	sell	768.0
        2021-01-19T14:45:00.000Z	AAPL	127.89	buy	782.0
        2021-01-26T12:15:00.000Z	AAPL	142.405	sell	702.0

Example output from MSFT part of csv file:

        datetime                        symbol  close   signal  shares

        2021-01-04T12:00:00.000Z	MSFT	215.09	sell	465.0
        2021-01-05T14:45:00.000Z	MSFT	217.64	buy	459.0
        2021-01-06T09:45:00.000Z	MSFT	213.2406	sell	469.0
        2021-01-07T11:00:00.000Z	MSFT	217.64	buy	459.0
        2021-01-08T13:15:00.000Z	MSFT	217.75999	sell	459.0
        2021-01-08T15:15:00.000Z	MSFT	219.55	buy	455.0
        2021-01-11T11:15:00.000Z	MSFT	218.08	sell	459.0
        2021-01-13T11:45:00.000Z	MSFT	215.95	buy	463.0
        2021-01-14T11:45:00.000Z	MSFT	214.6358	sell	466.0
        2021-01-19T10:00:00.000Z	MSFT	214.50999	buy	466.0
        2021-01-19T10:30:00.000Z	MSFT	213.49001	sell	468.0
        2021-01-19T10:45:00.000Z	MSFT	213.87	buy	468.0

As you can see above, the csv file is formatted by datetime, stock symbol, close price, buy/sell signal, and then finally the number of shares, assuming you are buying or selling $100,000 in stocks.